<a href="https://colab.research.google.com/github/R-Madhuram/UC-BerkeleyCapstone_CreditCardFraudDetection/blob/main/notebooks/AutoML_ensembleModels/13_Auto_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto ML Model Using Pycaret

In [ ]:
!pip install --pre pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.1/480.1 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
import xgboost

In [ ]:
import pandas as pd
import numpy as np
import pickle

## Importing helper functions



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import the helper function files
# Importing the helper_functions file
!cp /content/drive/MyDrive/PCMALAI_UCBerkeley_Capstone/Notebooks/helper_functions.py helper_functions.py
!cp /content/drive/MyDrive/PCMALAI_UCBerkeley_Capstone/Notebooks/helper_functions_ml.py helper_functions_ml.py

In [ ]:
# Import the helper functions
# Import the required functions from the helper function file
from helper_functions_ml import createMetricsDF,computeModelMetrics,computeAndPlotMetrics

## Import the Data (training and testing)

In [ ]:
# Import training data
!cp '/content/drive/MyDrive/PCMALAI_UCBerkeley_Capstone/Classification_products/df.csv' 'df.csv'

In [ ]:
# Read the training data
df_train = pd.read_csv('df.csv')
print(df_train.head())

df_train = df_train.drop('Unnamed: 0', axis=1)
df_train.head(3)
print(df_train.shape)

   Unnamed: 0  TransactionID  isFraud  TransactionDT  TransactionAmt  \
0           0        2987000        0          86400        1.835691   
1           1        2987001        0          86401        1.462398   
2           2        2987002        0          86469        1.770852   
3           3        2987003        0          86499        1.698970   
4           4        2987004        0          86506        1.698970   

   ProductCD  card1  card2  card3  card4  ...  id_36  id_37  id_38  \
0          4  13926   -1.0  150.0      1  ...      2      2      2   
1          4   2755  404.0  150.0      2  ...      2      2      2   
2          4   4663  490.0  150.0      4  ...      2      2      2   
3          4  18132  567.0  150.0      2  ...      2      2      2   
4          1   4497  514.0  150.0      2  ...      0      1      1   

   DeviceType  DeviceInfo  P_emaildomain_addr1_card1  card1_card2  \
0           1        1735                      63363         3655   
1       

In [ ]:
# CP cannot be used to copy file here because numFolds is a variable and hence the filenames will change based on the number of folds.
# instead of using linux CP command we use SHUTIL copy 

# read the K fold train and dev indices

numFolds = 3

import shutil # file copy package. 

srcFileName = '/content/drive/MyDrive/PCMALAI_UCBerkeley_Capstone/Classification_products/train_dev_indices_' + str(numFolds) + '.pickle' 
train_dev_indices_file = 'train_dev_indices_' + str(numFolds) + '.pickle'
# !cp  srcFileName destFileName
shutil.copyfile(srcFileName, train_dev_indices_file)


'train_dev_indices_3.pickle'

In [ ]:
# read the train dev indices file
# train_dev_indices_df = pd.read_csv(train_dev_indices_file)
# train_dev_indices_df.head()

with open(train_dev_indices_file, 'rb') as handle:
    train_dev_indices = pickle.load(handle)
train_dev_indices

{'train': [array([195756, 195757, 195758, ..., 590537, 590538, 590539]),
  array([     0,      1,      2, ..., 590537, 590538, 590539]),
  array([     0,      1,      2, ..., 397410, 397417, 397418])],
 'dev': [array([     0,      1,      2, ..., 221151, 221172, 221182]),
  array([195756, 195757, 195758, ..., 397410, 397417, 397418]),
  array([393481, 393482, 393483, ..., 590537, 590538, 590539])]}

In [ ]:
# Getting dataready
X = df_train.drop(columns = ['isFraud','TransactionID'])
y = df_train['isFraud']

In [ ]:
# List of categorical and numerical columns in merged dataset

## Numerical columns

num_column_lst_final = []
for icol in X.columns:
  if X[icol].dtypes in ['float64', 'int64']:
    num_column_lst_final.append(icol)
print(f"The columns that have numerical features are: '{num_column_lst_final}'")
print(f"The number of columns that have numerical features are: '{len(num_column_lst_final)}'")

print('\n')

## Categorical Columns
cat_column_lst_final = []
for icol in X.columns:
  if X[icol].dtypes in ['object']:
    cat_column_lst_final.append(icol)
print(f"The columns that have categorical features are: '{cat_column_lst_final}'")
print(f"The number of columns that have categorical features are: '{len(cat_column_lst_final)}'")


The columns that have numerical features are: '['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C3', 'C5', 'D1', 'D3', 'D4', 'D8', 'D9', 'D10', 'D13', 'D14', 'M1', 'M2', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V4', 'V6', 'V10', 'V12', 'V14', 'V15', 'V19', 'V23', 'V25', 'V27', 'V35', 'V37', 'V39', 'V44', 'V46', 'V53', 'V55', 'V61', 'V66', 'V75', 'V77', 'V82', 'V86', 'V95', 'V98', 'V99', 'V104', 'V107', 'V108', 'V109', 'V117', 'V118', 'V120', 'V121', 'V123', 'V124', 'V129', 'V135', 'V138', 'V139', 'V141', 'V144', 'V148', 'V161', 'V169', 'V170', 'V172', 'V173', 'V174', 'V184', 'V194', 'V208', 'V209', 'V214', 'V220', 'V221', 'V223', 'V224', 'V226', 'V227', 'V228', 'V238', 'V240', 'V241', 'V250', 'V260', 'V270', 'V281', 'V282', 'V284', 'V286', 'V288', 'V290', 'V300', 'V305', 'V313', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08',

## Models

In [ ]:
from pycaret.classification import setup,create_model,tune_model

In [ ]:
exp = setup(data= df_train, target='isFraud',session_id=42,use_gpu=True)

,Description,Value
0,Session id,42
1,Target,isFraud
2,Target type,Binary
3,Original data shape,"(590540, 157)"
4,Transformed data shape,"(590540, 157)"
5,Transformed train set shape,"(413378, 157)"
6,Transformed test set shape,"(177162, 157)"
7,Numeric features,156
8,Preprocess,True
9,Imputation type,simple


### XGBoost Model

In [ ]:
# xgboost model
xgb = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9795,0.9377,0.4592,0.9108,0.6106,0.6012,0.6387
1,0.9802,0.9322,0.4703,0.9302,0.6247,0.6157,0.6538
2,0.9792,0.9304,0.4426,0.9209,0.5979,0.5885,0.6305
3,0.9797,0.9372,0.4564,0.9231,0.6108,0.6016,0.6413
4,0.9798,0.9359,0.4706,0.9068,0.6197,0.6103,0.6452
5,0.9792,0.9297,0.4471,0.9177,0.6013,0.5919,0.6326
6,0.9797,0.9379,0.4768,0.8926,0.6216,0.6122,0.6442
7,0.9793,0.9290,0.4520,0.9109,0.6042,0.5947,0.6336
8,0.9781,0.9290,0.4246,0.8937,0.5757,0.5659,0.6076


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 14.6 MB/s eta 0:00:00


### LightGBM Model

In [ ]:
exp = setup(data= df_train, target='isFraud',session_id=42,use_gpu=True)
lightgbm = create_model('lightgbm')

,Description,Value
0,Session id,42
1,Target,isFraud
2,Target type,Binary
3,Original data shape,"(590540, 157)"
4,Transformed data shape,"(590540, 157)"
5,Transformed train set shape,"(413378, 157)"
6,Transformed test set shape,"(177162, 157)"
7,Numeric features,156
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9775,0.9277,0.3990,0.9044,0.5537,0.5440,0.5925
1,0.9776,0.9252,0.4039,0.9040,0.5583,0.5486,0.5960
2,0.9766,0.9168,0.3838,0.8782,0.5342,0.5240,0.5718
3,0.9775,0.9282,0.4108,0.8826,0.5606,0.5507,0.5935
4,0.9769,0.9274,0.3953,0.8760,0.5448,0.5346,0.5797
5,0.9768,0.9155,0.3829,0.8907,0.5355,0.5255,0.5754
6,0.9766,0.9254,0.3912,0.8681,0.5393,0.5291,0.5738
7,0.9768,0.9189,0.3905,0.8814,0.5412,0.5311,0.5780
8,0.9761,0.9178,0.3707,0.8701,0.5199,0.5096,0.5591


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!pip install pycaret[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 KB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB

### CatBoost Model

In [ ]:
from pycaret.classification import setup,create_model,tune_model,stack_models

In [ ]:
exp = setup(data= df_train, target='isFraud',session_id=42,use_gpu=True)
catboost = create_model('catboost')

,Description,Value
0,Session id,42
1,Target,isFraud
2,Target type,Binary
3,Original data shape,"(590540, 157)"
4,Transformed data shape,"(590540, 157)"
5,Transformed train set shape,"(413378, 157)"
6,Transformed test set shape,"(177162, 157)"
7,Numeric features,156
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9757,0.8982,0.3437,0.9020,0.4977,0.4879,0.5485
1,0.9761,0.9026,0.3492,0.9182,0.5060,0.4963,0.5582
2,0.9754,0.8845,0.3333,0.9009,0.4866,0.4767,0.5397
3,0.9759,0.8993,0.3465,0.9093,0.5018,0.4919,0.5531
4,0.9763,0.9005,0.3635,0.8976,0.5175,0.5075,0.5628
5,0.9753,0.8871,0.3262,0.9077,0.4799,0.4701,0.5359
6,0.9752,0.8973,0.3428,0.8687,0.4916,0.4813,0.5368
7,0.9755,0.8903,0.3345,0.9064,0.4886,0.4788,0.5424
8,0.9748,0.8838,0.3181,0.8915,0.4689,0.4590,0.5241


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

### Stacked Classifier 1 (DT Meta model)

In [ ]:
dt = create_model('dt')
stacker_dt = stack_models(estimator_list = [lightgbm,xgb,catboost],meta_model=dt)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9660,0.7671,0.5533,0.5125,0.5321,0.5145,0.5149
1,0.9643,0.7696,0.5602,0.4912,0.5234,0.5050,0.5061
2,0.9656,0.7676,0.5546,0.5073,0.5299,0.5121,0.5126
3,0.9657,0.7683,0.5560,0.5092,0.5316,0.5138,0.5144
4,0.9668,0.7847,0.5888,0.5227,0.5538,0.5366,0.5376
5,0.9647,0.7669,0.5543,0.4963,0.5237,0.5054,0.5062
6,0.9649,0.7693,0.5591,0.4985,0.5270,0.5089,0.5098
7,0.9644,0.7634,0.5473,0.4922,0.5183,0.4999,0.5006
8,0.9646,0.7577,0.5353,0.4943,0.5139,0.4956,0.4960


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9650,0.7673,0.5546,0.5003,0.5261,0.5080,0.5087
1,0.9651,0.7680,0.5560,0.5009,0.5270,0.5090,0.5097
2,0.9641,0.7648,0.5505,0.4886,0.5177,0.4992,0.5001
3,0.9661,0.7701,0.5595,0.5137,0.5356,0.5180,0.5185
4,0.9660,0.7756,0.5708,0.5130,0.5404,0.5228,0.5236
5,0.9638,0.7548,0.5301,0.4839,0.5059,0.4872,0.4877
6,0.9641,0.7623,0.5453,0.4879,0.5150,0.4964,0.4972
7,0.9652,0.7655,0.5508,0.5022,0.5254,0.5073,0.5079
8,0.9645,0.7530,0.5256,0.4925,0.5085,0.4901,0.4904


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

### Stacked Classifier 2 (AdaBoost Meta model)

In [ ]:
ada = create_model('ada')
stacker_ada = stack_models(estimator_list = [lightgbm,xgb,catboost],meta_model=ada)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9703,0.8608,0.1936,0.8211,0.3134,0.3041,0.3902
1,0.9697,0.8600,0.1992,0.7559,0.3153,0.3051,0.3785
2,0.9700,0.8470,0.1826,0.8148,0.2983,0.2892,0.3773
3,0.9701,0.8633,0.2033,0.7757,0.3222,0.3122,0.3878
4,0.9705,0.8636,0.2163,0.7825,0.3389,0.3288,0.4020
5,0.9694,0.8494,0.1804,0.7654,0.2919,0.2824,0.3624
6,0.9695,0.8607,0.1907,0.7520,0.3043,0.2943,0.3692
7,0.9701,0.8519,0.2073,0.7673,0.3264,0.3163,0.3893
8,0.9694,0.8515,0.1867,0.7521,0.2992,0.2893,0.3653


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,05:00:36
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Stacking Classifier


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9799,0.9405,0.5401,0.8265,0.6533,0.6434,0.6589
1,0.9806,0.9303,0.5505,0.8414,0.6656,0.6560,0.6717
2,0.9801,0.9309,0.5277,0.8440,0.6494,0.6397,0.6584
3,0.9805,0.9397,0.5201,0.8694,0.6508,0.6414,0.6639
4,0.9800,0.9394,0.5529,0.8155,0.6590,0.6491,0.6621
5,0.9799,0.9322,0.5342,0.8312,0.6504,0.6406,0.6572
6,0.9801,0.9403,0.5356,0.8360,0.6529,0.6432,0.6601
7,0.9795,0.9319,0.5086,0.8440,0.6348,0.6249,0.6462
8,0.9785,0.9287,0.5076,0.8075,0.6234,0.6129,0.6305


### Stacked Classifier 3 (LightGBM Meta model)

In [ ]:
stacker_lightgbm = stack_models(estimator_list = [lightgbm,xgb,catboost],meta_model=lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9808,0.9432,0.5394,0.8581,0.6624,0.6531,0.6718
1,0.9812,0.9378,0.5539,0.8585,0.6734,0.6642,0.6811
2,0.9811,0.9347,0.5443,0.8648,0.6681,0.6589,0.6776
3,0.9813,0.9447,0.5422,0.8750,0.6695,0.6604,0.6805
4,0.9811,0.9429,0.5529,0.8556,0.6717,0.6624,0.6792
5,0.9805,0.9366,0.5335,0.8530,0.6565,0.6470,0.6659
6,0.9812,0.9432,0.5501,0.8643,0.6723,0.6631,0.6811
7,0.9806,0.9347,0.5238,0.8683,0.6534,0.6441,0.6659
8,0.9797,0.9344,0.5118,0.8457,0.6377,0.6278,0.6489


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

### Stacked Classifier 4 (Random Forest Meta model)

In [ ]:
rf = create_model('rf')
stacker_rf = stack_models(estimator_list = [lightgbm,xgb,catboost],meta_model=rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9802,0.9428,0.4585,0.9512,0.6188,0.6099,0.6531
1,0.9816,0.9416,0.4910,0.9673,0.6514,0.6430,0.6823
2,0.9810,0.9360,0.4786,0.9558,0.6378,0.6291,0.6692
3,0.9811,0.9416,0.4799,0.9586,0.6396,0.6310,0.6712
4,0.9816,0.9415,0.4976,0.9574,0.6548,0.6464,0.6832
5,0.9800,0.9368,0.4499,0.9545,0.6116,0.6026,0.6480
6,0.9805,0.9408,0.4720,0.9395,0.6283,0.6194,0.6584
7,0.9804,0.9333,0.4637,0.9518,0.6236,0.6148,0.6570
8,0.9797,0.9352,0.4433,0.9496,0.6044,0.5954,0.6414


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:02:49
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Stacking Classifier


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

## Result Summary

Auto-ML using PyCaret library provided by python is an attarctive option for complex machine learning tasks, since hyper-Parameter tuning is time consuming and computationally expensive for ensemble models and stacked meta models. 

We built the three best boosting models from our previous iterations namely, XGBoost, LightGBM, and CatBoost models and stacked them using various meta/base estimators (Decision tree, Ada Boost, LightGBM and Random Forest). 

The stacking classfiers with adaboost/ Light GBM meta-estimators presented thamselves with a ROC-AUC score of 0.94 and a recall score of 0.54. Hence we chose to deploy either adaboost or Light GBM to Azure cloud storage. 

These two models are also the best models that we have obtained in this project yeilding the highes ROC-AUC score. 

## Model Storage in Azure

In [ ]:
from pycaret.classification import finalize_model
final_ada = finalize_model(stacker_ada)
print(final_ada)

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['TransactionID', 'TransactionDT',
                                             'TransactionAmt', 'ProductCD',
                                             'card1', 'card2', 'card3', 'card4',
                                             'card5', 'card6', 'addr1', 'addr2',
                                             'dist1', 'dist2', 'P_emaildomain',
                                             'R_emaildomain', 'C1', 'C3', 'C5',
                                             'D1', 'D3', 'D4', 'D8', 'D9',
                                             'D10', 'D13', 'D14', '...
                                                               objective='binary:logistic',
                                                               predictor=None, ...)),
                                                ('CatBoost Cl

In [ ]:
from pycaret.classification import save_model,deploy_model, load_model
model_dir = '/content/drive/MyDrive/PCMALAI_UCBerkeley_Capstone/Classification_products/'
model_name = 'Final_StackedAda_model'

In [ ]:
save_model(final_ada, model_dir + model_name)

In [ ]:
final_ada = load_model(model_dir + model_name)

Transformation Pipeline and Model Successfully Loaded


In [ ]:
azure_connection_string = 'DefaultEndpointsProtocol=https;AccountName=madhuramdeployment;AccountKey=l1nkTN9+kIK4gj2OzvNR+kHhIK7sr4AtBe3g5mWjXn2Bn1sf3Rke6wwWyb0RDRcWoYs83hjkPr9z+AStLRuoHw==;EndpointSuffix=core.windows.net'

In [ ]:
import os
os.environ['AZURE_STORAGE_CONNECTION_STRING'] = azure_connection_string

In [ ]:
!pip install azure-storage-blob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
deploy_model(model = final_ada, model_name = model_name, platform = 'azure', authentication = {'container' : 'madhuramdeployment'})

Model Successfully Deployed on Azure Storage Blob


## Model Deployment in Local Host

In [ ]:
from pycaret.classification import create_api
create_api(final_ada, 'ada_api')

API successfully created. This function only creates a POST API, it doesn't run it automatically. To run your API, please run this command --> !python ada_api.py


In [ ]:
# create docker
from pycaret.classification import create_docker
create_docker('ada_api')

In [ ]:
from pycaret.classification import create_app
create_app(final_ada)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

## Future Work

We can further improve the prformance of these models through augumenting them with an unsupervised learning technique like Isolation forest for detecting anamalous observations. 

Also Deep Learning methods for tabular data can be utilized in combination with machine learning methods built above or alone to improve the overall perfomance of the model to acheive the business objective of identifying the fraud transactions. 